#pip install

In [1]:
!pip install rank-bm25 FlagEmbedding pythainlp

# Import lib

In [1]:
import pandas as pd
from FlagEmbedding import BGEM3FlagModel
from rank_bm25 import BM25Okapi
from pythainlp.tokenize import word_tokenize
import numpy as np
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import re 
import math
import torch
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt 

# DATA 

In [2]:
questiondf = pd.read_csv("data/Question_df.csv")
Question = questiondf['Question']
Question = Question.to_list()
print(Question)

['ในพระราชบัญญัตินี้ผู้ตรวจการ  หมายถึงอะไร', 'ถ้าฟิลม์รถยนต์มืดมากเกินไป', 'รถคนปกติใช้ไซเรนจะผิดไหม', 'เราบีบเเตรเพื่อที่จะสร้างสีสันได้หรือไม่', 'บันทุกของเกินความยาวของรถได้กี่เมตร', 'รถบันทุกคน จะต้องทำเรื่องอะไรไหม', 'การบันทุกคนจะต้องป้องกันหรือจัดการอะไรไหม', 'มีอะไรกำหนดไหมว่าต้องทำตามครื่องหมายจราจร เเละ สัญญาณจราจร', 'ไฟจราจรสีเหลืองหมายถึงอะไร เเล้วจะต้องประทำตัวอย่างไรเมื่อเจอ', 'ไฟสัญญาณจราจรที่อยู่เหนือช่องรถคืออะไร', 'พนักงานเจ้าหน้าที่ใช้เสียงเป่านกหวีดยาวหนึ่งครั้ง เราต้องทำอย่างไร', 'ไฟสัญญาณจราจรมีการเปลี่ยนเเปลงได้ไหม', 'คนปกติสามารถเพิ่มไฟจราจรได้หรือไม่', 'ถ้าเราทำไฟจราจรเสียหายเราจะโดยอะไรไหม', 'สามารถบีบเเตรเพื่อที่จะเตือนคนเดินเท้าได้ไหม', 'ต้องขับขี่รถฝั่งไหน ระหว่างฝั่งซ้ายกันฝั่งขวา', 'ถ้ารถมีความเร็วมากเเล้วอยากที่จะเเซงต้องอยู่เลนไหน', 'ถ้าจะจอดรถต้องส่งสัญญาณก่อนจอดกี่เมตร', 'ไฟสัญญาณสีเหลืองที่อยู่หลังรถคืออะไร', 'เวลาขับรถสวนกันที่ถนนเเคบต้องปฎิบัติอย่างไร', 'เมื่อเจอถนนที่มีลูกศรชี้ไปทิศทางเดียวต้องทำอย่างไร', 'ถ้าอายุไม่ถึงในการขับรถ', 'เสพสารเสพติดจ

## Corpus

In [3]:
corpus_df = pd.read_csv("data/corpus_df.csv")
corpus = corpus_df['corpus']
corpus =  corpus.tolist()
print(corpus)

['มาตรา 1 - พระราชบัญญัติจราจรทางบก  พ.ศ. 2522', 'มาตรา 2 - พระราชบัญญัตินี้ให้ใช้บังคับเมื่อพ้นกำหนดเก้าสิบวันนับแต่วันประกาศในราชกิจจานุเบกษาเป็นต้นไป', 'มาตรา 3 - ให้ยกเลิก(1) พระราชบัญญัติจราจรทางบก พุทธศักราช 2477(2) พระราชบัญญัติจราจรทางบก แก้ไขเพิ่มเติม พุทธศักราช 2478(3) พระราชบัญญัติจราจรทางบก (ฉบับที่ 3) พุทธศักราช 2481(4) พระราชบัญญัติจราจรทางบก (ฉบับที่ 4) พ.ศ. 2508(5) ประกาศของคณะปฏิวัติ ฉบับที่ 59 ลงวันที่ 26 มกราคม  พ.ศ. 2515', "มาตรา 4 - ในพระราชบัญญัตินี้(1) การจราจร หมายความว่า  การใช้ทางของผู้ขับขี่  คนเดินเท้า หรือคนที่จูง ขี่ หรือไล่ต้อนสัตว์(2) ทาง หมายความว่า  ทางเดินรถ ช่องเดินรถ ช่องเดินรถประจำทาง ไหล่ทาง ทางเท้า ทางข้าม ทางร่วมทางแยก ทางลาด ทางโค้ง สะพาน และลานที่ประชาชนใช้ในการจราจร  และให้หมายความรวมถึงทางส่วนบุคคลที่เจ้าของยินยอมให้ประชาชนใช้ในการจราจรหรือที่เจ้าพนักงานจราจรได้ประกาศให้เป็นทางตามพระราชบัญญัตินี้ด้วย แต่ไม่รวมไปถึงทางรถไฟ(3) ทางเดินรถ หมายความว่า  พื้นที่ที่ทำไว้สำหรับการเดินรถไม่ว่าในระดับพื้นดิน ใต้หรือเหนือพื้นดิน(4) ช่องเดินรถ หมายความว่

# Model

## BGE_M3

In [5]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',use_fp16=True)
embeddings_1 = model.encode(Question,
                            batch_size=16,
                            max_length=8192,
                            )['dense_vecs']
embeddings_2 = model.encode(corpus)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity) #Dense score 

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

[[0.337  0.4722 0.3857 ... 0.497  0.4243 0.469 ]
 [0.3752 0.3457 0.3845 ... 0.4246 0.457  0.446 ]
 [0.3008 0.334  0.3098 ... 0.4285 0.4995 0.4695]
 ...
 [0.371  0.4802 0.4016 ... 0.4866 0.4631 0.515 ]
 [0.3362 0.4004 0.3867 ... 0.5376 0.5205 0.516 ]
 [0.4114 0.3823 0.4768 ... 0.6934 0.657  0.5464]]


In [6]:
most_similar_indices = np.argmax(similarity, axis=1)
BGEM3 = [corpus[i]for i in most_similar_indices]

In [16]:
file_path = "save_embedded/BGEM3_similar_score" #

with open(file_path,'wb') as file :
  pickle.dump(similarity,file)

## intfloat/multilingual-e5-base // change a bit

### lenght

In [7]:
lenghtquestion = 0
for i in Question:
  if i:
    lenghtquestion += len(i)
lenghtquestion

4102

In [8]:
lenghtcorpus = 0
for i in corpus:
  if i:
    lenghtcorpus += len(i)
lenghtcorpus

69084

### Init model ME5

In [9]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

In [10]:
model.training

False

In [11]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [12]:
input_queries = Question
input_passages = corpus

In [17]:
query_dict = tokenizer(input_queries, max_length=512, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
  query_outputs = model(**query_dict)
  query_embeddings = average_pool(query_outputs.last_hidden_state, query_dict['attention_mask'])
  query_embeddings = F.normalize(query_embeddings, p=2, dim=1)

In [18]:
file_path = "save_embedded/ME5_Question" #

with open(file_path,'wb') as file :
  pickle.dump(query_embeddings,file)

In [19]:
query_embeddings.shape

torch.Size([103, 768])

In [20]:
passsagecorpus = []
batch_size = 4  # Adjust as needed

for i in range(math.ceil(len(input_passages)/batch_size)):
    batch_passages = input_passages[i * batch_size:(i+1) * batch_size]
    passage_dict = tokenizer(batch_passages, max_length=512, padding=True, truncation=True, return_tensors='pt')
    passage_outputs = model(**passage_dict)
    passage_embeddings = average_pool(passage_outputs.last_hidden_state, passage_dict['attention_mask'])
    passsagecorpus.append(passage_embeddings)
passsage_embeddings = torch.cat(passsagecorpus, dim=0)
passsage_embeddings_normalized = F.normalize(passsage_embeddings, p=2, dim=1)

In [24]:
file_path = "save_embedded/ME5_Corpus" #

with open(file_path,'wb') as file :
  pickle.dump(passsage_embeddings_normalized,file)


In [25]:
passsage_embeddings_normalized.shape

torch.Size([172, 768])

## loaddata from save

In [26]:
with open("save_embedded/ME5_Question",'rb') as file :
  question_tensor = pickle.load(file)

with open("save_embedded/ME5_Corpus",'rb') as file :
  corpus_tensor = pickle.load(file)

question_tensor.shape , corpus_tensor.shape

(torch.Size([103, 768]), torch.Size([172, 768]))

In [27]:
def calsimilarity(query_embeddings,passage_embeddings):
  scores = (query_embeddings @ passage_embeddings.T)
  return scores

In [38]:
score = calsimilarity(question_tensor,corpus_tensor)
score.shape

torch.Size([103, 172])

In [46]:
file_path = "save_embedded/ME5_similar_score" #

with open(file_path,'wb') as file :
  pickle.dump(score,file)

In [93]:
index = torch.argmax(score,dim=-1)

In [94]:
ME5 = [corpus[i]for i in index]

# BM25

In [47]:
query = "คำถามเกี่ยวกับผู้ขับขี่ที่ไม่สวมหมวกกันน็อคขณะขับขี่รถจักรยานยนต์เข้าข่ายมาตราใด?"

In [48]:
word_tokenize(query, engine="newmm")

['คำถาม',
 'เกี่ยวกับ',
 'ผู้ขับขี่',
 'ที่',
 'ไม่',
 'สวม',
 'หมวก',
 'กัน',
 'น็อค',
 'ขณะ',
 'ขับขี่',
 'รถจักรยานยนต์',
 'เข้าข่าย',
 'มาตรา',
 'ใด',
 '?']

In [87]:
Questionlist = []
BM25 = []
tokenized_corpus = [word_tokenize(doc, engine="newmm") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [88]:
for i in Question:
  query = i
  tokenized_query = word_tokenize(query, engine="newmm")
  doc_scores = bm25.get_scores(tokenized_query)
  ans = bm25.get_top_n(tokenized_query,corpus, n=1)
  print(doc_scores)
  Questionlist.append(query)
  BM25.append(ans)
  print(query,ans)

[5.27656006 6.55886762 4.9005602  7.25459519 6.96670578 1.57582484
 1.20926404 3.70992643 1.42667681 1.65254301 3.5719153  1.95625227
 1.86562147 1.81955223 1.80568498 1.85222249 1.21639355 1.76954847
 1.72148618 1.30075152 1.58144649 1.81096817 1.45118717 1.67837892
 1.55095454 1.2022176  1.90214511 6.5824122  1.36984227 1.30900431
 1.26100083 3.8039372  1.55889554 1.71559569 1.9356655  1.67784547
 0.72490189 1.06733623 0.79470771 1.42359893 1.6658863  0.
 0.         1.75456208 5.86370549 3.78346285 7.98533986 1.25334047
 1.25919611 1.69214443 1.61699284 1.629766   1.82700489 1.31959957
 0.         2.81793708 1.86555173 1.75450039 1.59360681 1.97268087
 1.97642688 4.36031221 1.65918783 6.50530383 1.69322994 2.01897689
 1.34981647 1.55889554 1.62648756 1.85844909 6.07664209 1.84332816
 0.         3.40657078 2.10263055 1.81237608 0.         1.77912494
 1.59505134 3.76282014 2.44886355 1.83241376 1.13455627 1.49924762
 1.64114294 1.61376554 0.         2.57866191 2.56552435 0.
 1.08809127

In [98]:
BM25_0 = []
for score_dict in BM25:
    BM25_0.extend(score_dict)

# Eval

In [4]:
answer = pd.read_csv("data/Answer.csv")
def Eval_andremove(df,Answer_df,name):
    performance  = 0
    Predict = df
    pattern  = r"([^-\n]+) - "
    remove = []
    for i in Predict:
        matches = re.findall(pattern,i)
        remove.append(matches)
    Final_df = pd.DataFrame({"predict":remove})
    Final_df = Final_df['predict'].str[0].str.replace(r"\[\['|'\]", '', regex=True).replace(r"^\['", '', regex=True)
    for index in range(len(Final_df)): 
        if Final_df.iloc[index] == Answer_df["REALANS"].iloc[index]:
            performance += 1
    print(performance)
    print(f"{name} output score : {performance} | % = {performance/103}")
    return performance

In [99]:
BGE_M3_score = Eval_andremove(BGEM3,answer,"BGE_M3") #best score 
ME5_score = Eval_andremove(ME5,answer,"ME5")
BM25_score = Eval_andremove(BM25_0,answer,"BM25")

64
BGE_M3 output score : 64 | % = 0.6213592233009708
52
ME5 output score : 52 | % = 0.5048543689320388
27
BM25 output score : 27 | % = 0.2621359223300971


# BGE M3 ALL score 

In [5]:
Question_df = pd.read_csv("data/Question_df.csv")
Question_list = Question_df['Question']
Question_list

0              ในพระราชบัญญัตินี้ผู้ตรวจการ  หมายถึงอะไร
1                             ถ้าฟิลม์รถยนต์มืดมากเกินไป
2                               รถคนปกติใช้ไซเรนจะผิดไหม
3               เราบีบเเตรเพื่อที่จะสร้างสีสันได้หรือไม่
4                    บันทุกของเกินความยาวของรถได้กี่เมตร
                             ...                        
98              เงินที่โดยปรับจะเก็บเข้าหน่วยงานอะรไบ้าง
99                  ถ้าผิดตามมาตรา 106 ต้องโดนปรับกี่บาท
100                   ถ้าผิดตามมาตรา 8 ต้องโดนปรับกี่บาท
101    ถ้าเเท็กซี่รับรถที่ไม่ใช่ตามพระราชบัญญัติกำหนด...
102                     ถ้าทำผิดมีโอกาสโดนยึดใบขับขี่ไหม
Name: Question, Length: 103, dtype: object

In [6]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
ALL_final_ans_list_ALL = []
ALL_final_ans_list_Sparse_Dense = []
ALL_final_ans_list_sparse = []
ALL_final_ans_list_score_colbert = []
ALL_final_ans_list_Dense = []
 
final_ans_list_ALL = []
final_ans_list_Sparse_Dense = []
final_ans_list_sparse = []
final_ans_list_score_colbert = []
final_ans_list_Dense = []
batch_size = 10  # Adjust based on your GPU memory capacity



All_loop_params = tqdm(range(len(Question_list)))
for round in All_loop_params:
    questions = Question_list[round]
    sentence_pairs = [[questions, j] for j in corpus]
    
    listofscore = []
    compute_Score = range(0, len(sentence_pairs), batch_size)

    for i in compute_Score:
        batch_pairs = sentence_pairs[i:i+batch_size]
        allscore = model.compute_score(batch_pairs,
                                       max_passage_length=512,  # A smaller max length leads to a lower latency
                                       weights_for_different_modes=[0.4, 0.2, 0.4]) # sum: w[0]*dense_score + w[1]*sparse_score + w[2]*colbert_score
        listofscore.append(allscore)  # Convert to list and move to CPU

    score_ALL = []
    score_sparse_dense = []
    score_colbert = []
    score_dense = []
    score_sparse = []
    
    for score_dict in listofscore:
        score_dense.extend(score_dict['dense'])

    for score_dict in listofscore:
        score_sparse_dense.extend(score_dict['sparse+dense'])
    
    for score_dict in listofscore:
        score_colbert.extend(score_dict['colbert'])

    for score_dict in listofscore:
        score_sparse.extend(score_dict['sparse'])
    
    for score_dict in listofscore:
        score_ALL.extend(score_dict['colbert+sparse+dense'])
    
    index_ALL = np.argmax(score_ALL)
    final_ans_list_ALL.append(corpus[index_ALL])
    ALL_final_ans_list_ALL.append(score_ALL)

    index_Sparse_Dense = np.argmax(score_sparse_dense)
    final_ans_list_Sparse_Dense.append(corpus[index_Sparse_Dense])
    ALL_final_ans_list_Sparse_Dense.append(score_sparse_dense)
    
    index_ALL_sparse = np.argmax(score_sparse)
    final_ans_list_sparse.append(corpus[index_ALL_sparse])
    ALL_final_ans_list_sparse.append(score_sparse)

    index_ALL_score_colbert = np.argmax(score_colbert)
    final_ans_list_score_colbert.append(corpus[index_ALL_score_colbert])
    ALL_final_ans_list_score_colbert.append(score_colbert)


    index_ALL_dense = np.argmax(score_dense)
    final_ans_list_Dense.append(corpus[index_ALL_dense])
    ALL_final_ans_list_Dense.append(score_dense)


100%|██████████| 103/103 [07:44<00:00,  4.51s/it]


In [9]:
AllOutput_df_score = Eval_andremove(final_ans_list_ALL,answer,"ALL")
Multi_vec_Output_score = Eval_andremove(final_ans_list_score_colbert,answer,"Multivec")
Sparse_Dense_Output_score = Eval_andremove(final_ans_list_Sparse_Dense,answer,"Sparse+Dense")
SparseOutput_score = Eval_andremove(final_ans_list_sparse,answer,"Sparse")
DenseOutput_df_score = Eval_andremove(final_ans_list_Dense,answer,"Dense")

67
ALL output score : 67 | % = 0.6504854368932039
63
Multivec output score : 63 | % = 0.6116504854368932
64
Sparse+Dense output score : 64 | % = 0.6213592233009708
44
Sparse output score : 44 | % = 0.42718446601941745
64
Dense output score : 64 | % = 0.6213592233009708


# Reranker 

In [8]:
ALL_final_ans_list_Dense

[[0.3369140625,
  0.472412109375,
  0.385986328125,
  0.52099609375,
  0.51953125,
  0.465576171875,
  0.39990234375,
  0.46044921875,
  0.40576171875,
  0.392333984375,
  0.43603515625,
  0.405029296875,
  0.3935546875,
  0.451171875,
  0.399658203125,
  0.369140625,
  0.349609375,
  0.408447265625,
  0.35693359375,
  0.396484375,
  0.433349609375,
  0.459228515625,
  0.39501953125,
  0.42578125,
  0.443115234375,
  0.409912109375,
  0.42822265625,
  0.45556640625,
  0.483642578125,
  0.4541015625,
  0.42333984375,
  0.344482421875,
  0.402587890625,
  0.35986328125,
  0.368896484375,
  0.3916015625,
  0.3876953125,
  0.356689453125,
  0.385986328125,
  0.362060546875,
  0.368408203125,
  0.43896484375,
  0.4013671875,
  0.452392578125,
  0.5166015625,
  0.56396484375,
  0.51171875,
  0.50390625,
  0.36376953125,
  0.337646484375,
  0.372314453125,
  0.42041015625,
  0.391357421875,
  0.406005859375,
  0.40771484375,
  0.358154296875,
  0.3828125,
  0.4013671875,
  0.369140625,
  0.39

In [10]:
topkindex = 15
ALL_final_ans_list_Dense_np = np.array(ALL_final_ans_list_Dense)
topk15indices = np.argsort(ALL_final_ans_list_Dense_np, axis=1)[:, -topkindex:]
topk15indices

array([[ 63, 162, 151, ..., 150, 149,  45],
       [ 84,   7,  16, ...,  38,  65,  11],
       [ 12,  80, 131, ...,  79,  51,  13],
       ...,
       [168, 166, 162, ..., 167, 156, 154],
       [ 44, 169, 148, ..., 146, 162, 105],
       [160, 145, 128, ..., 164, 170, 169]], dtype=int64)

In [11]:
len(topk15indices)

103

In [12]:
corpus_length = len(corpus_df)  # Ensure corpus length is known

topkindex = 15

Input_text_forrerank = []

for indices in topk15indices:
    containptopk = []
    for i, index in enumerate(indices):
        if index < corpus_length:  # Check if index is valid for corpus
            containptopk.append(' '.join(map(str, corpus_df.iloc[index].tolist())))  # Convert list to string and join elements
        if i >= topkindex - 1:
            Input_text_forrerank.append(containptopk)
            containptopk = []

print(Input_text_forrerank)


[['มาตรา 59 - เจ้าพนักงานจราจรหรือพนักงานเจ้าหน้าที่มีอำนาจสั่งให้ผู้ขับขี่เคลื่อนย้ายรถที่หยุดหรือจอดอยู่อันเป็นการฝ่าฝืนบทแห่งพระราชบัญญัตินี้ได้เจ้าพนักงานจราจรหรือพนักงานเจ้าหน้าที่มีอำนาจเคลื่อนย้ายรถที่หยุดหรือจอดอยู่อันเป็นการฝ่าฝืนบทแห่งพระราชบัญญัตินี้ หรือใช้เครื่องมือบังคับไม่ให้เคลื่อนย้ายรถดังกล่าวได้การเคลื่อนย้ายรถหรือใช้เครื่องมือบังคับให้รถที่หยุดหรือจอดอยู่ไม่ให้เคลื่อนย้ายได้ตามวรรคสอง เจ้าพนักงาน-จราจรหรือพนักงานเจ้าหน้าที่ไม่ต้องรับผิดสำหรับความเสียหายใด ๆ ที่เกิดขึ้นจากการปฏิบัติตามมาตรานี้ เว้นแต่ความเสียหายนั้นจะเกิดขึ้นจากการกระทำโดยจงใจหรือประมาทเลินเล่อเจ้าของรถหรือผู้ขับขี่ต้องชำระค่าใช้จ่ายในการที่รถถูกเคลื่อนย้ายหรือถูกใช้เครื่องมือบังคับไม่ให้เคลื่อนย้าย ตลอดจนค่าดูแลรักษารถระหว่างที่อยู่ในความครอบครองของเจ้าพนักงานจราจรหรือพนักงานเจ้าหน้าที่  ทั้งนี้ตามอัตราที่กำหนดในกฎกระทรวง ซึ่งต้องกำหนดอัตราค่าใช้จ่ายไม่น้อยกว่าคันละห้าร้อยบาทและค่าดูแลรักษาไม่น้อยกว่าวันละสองร้อยบาท', 'มาตรา 156 - ผู้ใดนำรถที่เจ้าพนักงานจราจร พนักงานเจ้าหน้าที่ หรือผู้ตรวจการได้สั่ง

## Normal reranker 

In [13]:
from FlagEmbedding import FlagReranker
rerankerbase = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)  # Setting use_fp16 to True speeds up computation with a slight performance degradation

# Initialize variables
result_df = []
i = 0

print(Question_list)

for index in range(len(Question_list)):
    question = Question_list[index]
    Input_text_forrerank_round = Input_text_forrerank[index]
    scores_base = []

    # Iterate through each passage in the final answers list
    for passage in Input_text_forrerank_round:
        passage = str(passage)
        score_base = rerankerbase.compute_score([question, passage], normalize=True)
        scores_base.append(score_base)
        print(passage[:20])

    # Find the highest scoring passage
    highest_scoring_index = scores_base.index(max(scores_base))
    highest_scoring_passage = Input_text_forrerank_round[highest_scoring_index]

    print(f"Highest: {max(scores_base)} - '{highest_scoring_passage}'")
    result_df.append(highest_scoring_passage)

print(result_df)


0              ในพระราชบัญญัตินี้ผู้ตรวจการ  หมายถึงอะไร
1                             ถ้าฟิลม์รถยนต์มืดมากเกินไป
2                               รถคนปกติใช้ไซเรนจะผิดไหม
3               เราบีบเเตรเพื่อที่จะสร้างสีสันได้หรือไม่
4                    บันทุกของเกินความยาวของรถได้กี่เมตร
                             ...                        
98              เงินที่โดยปรับจะเก็บเข้าหน่วยงานอะรไบ้าง
99                  ถ้าผิดตามมาตรา 106 ต้องโดนปรับกี่บาท
100                   ถ้าผิดตามมาตรา 8 ต้องโดนปรับกี่บาท
101    ถ้าเเท็กซี่รับรถที่ไม่ใช่ตามพระราชบัญญัติกำหนด...
102                     ถ้าทำผิดมีโอกาสโดนยึดใบขับขี่ไหม
Name: Question, Length: 103, dtype: object
มาตรา 59 - เจ้าพนักง
มาตรา 156 - ผู้ใดนำร
มาตรา 145 - บรรดาควา
มาตรา 161 (4) - ในกร
มาตรา 154 - ผู้ใด(1)
มาตรา 43 เบญจ(4) - ใ
มาตรา 157 ทวิ(5) - ผ
มาตรา 43 จัตวา(3) - 
มาตรา 43 ทวิ(1) - ห้
มาตรา 142 - เจ้าพนัก
มาตรา 5 - ให้รัฐมนตร
มาตรา 4 - ในพระราชบั
มาตรา 144 - เมื่อเจ้
มาตรา 143 ทวิ(5) - เ
มาตรา 43 ตรี(2) - ใน
Highest: 0.3688

In [14]:
len(result_df)
rerankscore = Eval_andremove(result_df,answer,"ALL")

70
ALL output score : 70 | % = 0.6796116504854369


## LLM based reranker

In [ ]:
from FlagEmbedding import FlagLLMReranker
rerankerLLM = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', use_fp16=True)  

result_df_LLM = []

print(Question_list)

for index in tqdm(range(len(Question_list))):
    question = Question_list[index]
    Input_text_forrerank_round = Input_text_forrerank[index]
    scores_base = []

    # Iterate through each passage in the final answers list
    for passage in Input_text_forrerank_round:
        passage = str(passage)
        score_base = rerankerLLM.compute_score([question, passage], normalize=True)
        scores_base.append(score_base)
        print(passage[:20])

    # Find the highest scoring passage
    highest_scoring_index = scores_base.index(max(scores_base))
    highest_scoring_passage = Input_text_forrerank_round[highest_scoring_index]

    print(f"Highest: {max(scores_base)} - '{highest_scoring_passage}'")
    result_df_LLM.append(highest_scoring_passage)

print(result_df_LLM)


In [ ]:
len(result_df_LLM)
rerankscore_LLM = Eval_andremove(result_df_LLM,answer,"ALL")

## LLM based layerwise 

In [ ]:
from FlagEmbedding import LayerWiseFlagLLMReranker
rerankerlayer = LayerWiseFlagLLMReranker('BAAI/bge-reranker-v2-minicpm-layerwise', use_fp16=True)

result_df_layer = []
i = 0

print(Question_list)

for index in tqdm(range(len(Question_list))):
    question = Question_list[index]
    Input_text_forrerank_round = Input_text_forrerank[index]
    scores_base = []

    # Iterate through each passage in the final answers list
    for passage in Input_text_forrerank_round:
        passage = str(passage)
        score_base = rerankerlayer.compute_score([question, passage], cutoff_layers=[28])
        scores_base.append(score_base)
        print(passage[:20])

    # Find the highest scoring passage
    highest_scoring_index = scores_base.index(max(scores_base))
    highest_scoring_passage = Input_text_forrerank_round[highest_scoring_index]

    print(f"Highest: {max(scores_base)} - '{highest_scoring_passage}'")
    result_df_layer.append(highest_scoring_passage)

print(result_df_layer)


In [ ]:
len(result_df_layer)
rerank_score_layerwise = Eval_andremove(result_df_layer,answer,"ALL")